In [90]:
# Importar bibliotecas necessárias
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import nltk

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
# from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import LabelEncoder
from tpot import TPOTClassifier
import os
import pickle
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
import seaborn as sns

#Importando modelo Dummy:
from sklearn.dummy import DummyClassifier



## Pré-processamento

### Lendo o arquivo

In [91]:
# Ler o arquivo CSV
df = pd.read_csv("../Lucas Braz/data.csv", names=['data', 'label'])

### Removendo linhas com valores nulos ou vazios

In [92]:
# Remover linhas com valores nulos ou vazios
df = df.dropna()

### Removendo a primeira linha

In [93]:
# Remover a primeira linha
df = df.iloc[1:]


### Removendo alphanumericos, pontuação, stopwords e deixando tudo em minúsculo

In [94]:
nltk.download('stopwords')
# Remover alfanuméricos
df['data'] = df['data'].str.replace('\d+', '')

# Deixar tudo em minúsculo
df['data'] = df['data'].str.lower()

# Remover pontuação
df['data'] = df['data'].str.replace('[^\w\s]', '')

# Remover stopwords
stopwords = nltk.corpus.stopwords.words('english')
df['data'] = df['data'].apply(lambda x: " ".join(x for x in x.split() if x not in stopwords))

[nltk_data] Downloading package stopwords to /home/apo-pc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/tmp/ipykernel_21473/896700526.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['data'] = df['data'].str.replace('\d+', '')
/tmp/ipykernel_21473/896700526.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['data'] = df['data'].str.replace('[^\w\s]', '')


### Lemmatização

In [95]:
# Lemmatização
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
df['data'] = df['data'].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split()]))

[nltk_data] Downloading package wordnet to /home/apo-pc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Dividindo os dados em treino e teste

In [96]:
# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(df['data'], df['label'], test_size=0.2, random_state=42)

In [97]:
# COntar qtd de classes
y_train.value_counts()


neutral     2508
positive    1480
negative     685
Name: label, dtype: int64

### Codificando a variável target

In [98]:
# Instantiate the label encoder
label_encoder = LabelEncoder()

# Fit and transform the training labels
y_train = label_encoder.fit_transform(y_train)

# Transform the test labels (use the same label encoder instance)
y_test = label_encoder.transform(y_test)

## Vetorização
### TF-IDF

In [99]:
# Pré-processar os dados de texto
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)



X_puro_train_tfidf = X_train_tfidf
X_puro_test_tfidf = X_test_tfidf

## Treinamento
- SVM
- Random Forest
- XGBoost
- MLP
- Naive Bayes

### Testando Modelos Classicos (SVM, Random Forest, XGBoost, MLP, Naive Bayes)

#### Sem GridSearch

In [100]:
def avalia_modelo_cv(model, X_input, y_input, n, resultados_individuais=False):
    kf = KFold(n_splits=n)
    reports = []
    y_true = []
    y_pred_list = []
    accuracy_list = []
    model_name = type(model).__name__
    for fold_idx, (train_index, test_index) in enumerate(kf.split(X_input)):
        X_train, X_test = X_input[train_index], X_input[test_index]
        y_train, y_test = y_input[train_index], y_input[test_index]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_true.extend(y_test)
        y_pred_list.append(y_pred)
        report = classification_report(y_test, y_pred, zero_division=0)
        reports.append(report)
        accuracy_list.append(accuracy_score(y_test, y_pred))
        if resultados_individuais:
            print(f"Classification Report for fold {fold_idx + 1}:\n{report}\n")
    y_pred_total = np.concatenate(y_pred_list)
    print("-" * 45)
    print("model_name: ", model_name)
    print("Average Classification Report:")
    avg_report = classification_report(y_true, y_pred_total, zero_division=0)
    print(avg_report)
    #Retorna a acuracia media
    return accuracy_score(y_true, y_pred_total)

# Função que chama os 4 modelos e retorna o resultado de cada um usando a função avalia_modelo_cv e imprime o resultado de cada um.

def avalia_dif_modelos_cv(X_train, y_test, cv):
    # Criando os modelos
    dummy_av = DummyClassifier(strategy='most_frequent')
    rf_av = RandomForestClassifier(n_jobs=-1)
    svm_av = SVC(kernel='rbf', gamma='auto', decision_function_shape='ovo')
    mlp_av = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=1000)
    xgb_av = XGBClassifier(n_jobs=-1)
    nb_av = DummyClassifier(strategy='stratified')

    # Aplicando a função de avaliação
    dummy_fun_av = avalia_modelo_cv(dummy_av, X_train, y_test, cv)
    rf_fun_av = avalia_modelo_cv(rf_av, X_train, y_test, cv)
    svm_fun_av = avalia_modelo_cv(svm_av, X_train, y_test, cv)
    mlp_fun_av = avalia_modelo_cv(mlp_av, X_train, y_test, cv)
    xgb_fun_av = avalia_modelo_cv(xgb_av, X_train, y_test, cv)

    # Criando o dataframe com os resultados
    df_resultado = pd.DataFrame({'Modelo': ['Dummy', 'RF', 'SVM', 'MLP', 'XGB', 'Naive Bayes'],
                                 'Acurácia': [dummy_fun_av, rf_fun_av, svm_fun_av, mlp_fun_av, xgb_fun_av, nb_fun_av]})

    # Plotando o gráfico de barras, que vai de 0 a 1
    sns.barplot(x='Modelo', y='Acurácia', data=df_resultado)
    plt.ylim(0, 1)
    plt.show("Desempenho dos modelos")
    %matplotlib inline

    return df_resultado

In [101]:
# Avaliando os modelos
# Concatenando os dados de treino e teste
X_total_tfidf = np.concatenate((X_train_tfidf.toarray(), X_test_tfidf.toarray()))
y_total = np.concatenate((y_train, y_test))
avalia_dif_modelos_cv(X_total_tfidf,y_total, 5)

---------------------------------------------
model_name:  DummyClassifier
Average Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       860
           1       0.54      1.00      0.70      3130
           2       0.00      0.00      0.00      1852

    accuracy                           0.54      5842
   macro avg       0.18      0.33      0.23      5842
weighted avg       0.29      0.54      0.37      5842
---------------------------------------------
model_name:  RandomForestClassifier
Average Classification Report:
              precision    recall  f1-score   support

           0       0.21      0.16      0.18       860
           1       0.67      0.78      0.72      3130
           2       0.73      0.61      0.67      1852

    accuracy                           0.63      5842
   macro avg       0.54      0.52      0.52      5842
weighted avg       0.62      0.63      0.62      5842
---------------------

NameError: name 'nb_fun_av' is not defined

#### Com GridSearch

##### GRIDSEARCH PARA O MAX_FEATURES DO TFIDF

In [ ]:
from sklearn.pipeline import Pipeline


pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('clf', XGBClassifier(n_jobs=-1)),
])
parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75, 1.0),
    'tfidf__max_features': (1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000),
    'tfidf__min_df': (0.01, 0.05, 0.1, 0.2, 0.3, 0.6, 0.9, 1.0),
    'tfidf__ngram_range': ((1, 1), (1, 2), (1, 3)),
}

grid_search_tune = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=3)
grid_search_tune.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_tune.best_estimator_.steps)

Fitting 5 folds for each of 768 candidates, totalling 3840 fits
[CV 2/5] END tfidf__max_df=0.25, tfidf__max_features=1000, tfidf__min_df=0.01, tfidf__ngram_range=(1, 2);, score=0.574 total time=   2.5s
[CV 1/5] END tfidf__max_df=0.25, tfidf__max_features=1000, tfidf__min_df=0.05, tfidf__ngram_range=(1, 1);, score=0.539 total time=   0.6s
[CV 2/5] END tfidf__max_df=0.25, tfidf__max_features=1000, tfidf__min_df=0.05, tfidf__ngram_range=(1, 1);, score=0.527 total time=   0.7s
[CV 3/5] END tfidf__max_df=0.25, tfidf__max_features=1000, tfidf__min_df=0.05, tfidf__ngram_range=(1, 1);, score=0.532 total time=   0.6s
[CV 1/5] END tfidf__max_df=0.25, tfidf__max_features=1000, tfidf__min_df=0.05, tfidf__ngram_range=(1, 2);, score=0.535 total time=   0.8s
[CV 4/5] END tfidf__max_df=0.25, tfidf__max_features=1000, tfidf__min_df=0.05, tfidf__ngram_range=(1, 3);, score=0.533 total time=   1.0s
[CV 2/5] END tfidf__max_df=0.25, tfidf__max_features=1000, tfidf__min_df=0.1, tfidf__ngram_range=(1, 3);, sc

##### GRIDSEARCH

In [48]:
# Definindo parametros:
# Definir os modelos e os grids de parâmetros
models = {
    'Random Forest': RandomForestClassifier(n_jobs=-1),
    'SVM': SVC(),
    'MLP': MLPClassifier(),
    'XGBoost': XGBClassifier(n_jobs=-1),
    'Naive Bayes': DummyClassifier(strategy='stratified')
}

param_grids = [
    {
        'model': 'Random Forest',
        'params': {
            'n_estimators': [50, 100, 200, 300, 400, 500],
            'max_depth': [5, 10, 15, 20],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['auto', 'sqrt', 'log2']
        }
    },
    {
        'model': 'SVM',
        'params': {
            'C': [0.1, 1, 10, 100],
            'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
            'gamma': ['scale', 'auto']
        }
    },
    {
        'model': 'MLP',
        'params': {
            'hidden_layer_sizes': [(10,), (50,), (100,), (150,), (200,)],
            'activation': ['logistic', 'tanh', 'relu'],
            'alpha': [0.001, 0.01, 0.1, 1],
            'learning_rate': ['constant', 'invscaling', 'adaptive']
        }
    },
    {
        'model': 'XGBoost',
        'params': {
            'n_estimators': [50, 100, 200, 400, 500],
            'max_depth': [3, 5, 7, 9, 11],
            'learning_rate': [0.01, 0.1, 0.3],
            'subsample': [0.6, 0.8, 1.0],
            'colsample_bytree': [0.6, 0.8, 1.0]
        }
    },
    {
        'model': 'Naive Bayes',
        'params': {
            'strategy': ['stratified', 'most_frequent', 'prior', 'uniform']
        }
    }
    
]



In [49]:
def perform_model_tuning(data, target, models, param_grids, base_name):
    # Dividir os dados em treinamento e teste após o Random Under-Sampling
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

    best_models = {}  # Dicionário para armazenar o melhor modelo de cada tipo
    model_accuracies = {}  # Dicionário para armazenar as acurácias de cada modelo
    current_dir = os.getcwd()
    for model_name, model, param_grid_entry in zip(models.keys(), models.values(), param_grids):
        print(f"Tuning do modelo {model_name} iniciado...")

        # Acessar o dicionário de parâmetros correspondente ao modelo atual
        param_grid = param_grid_entry['params']

        # Realizar a busca em grid para encontrar os melhores parâmetros
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
        grid_search.fit(X_train, y_train)

        # Melhor modelo encontrado
        best_model = grid_search.best_estimator_

        # Fazer previsões no conjunto de teste
        y_pred = best_model.predict(X_test)

        # Calcular a acurácia do modelo
        accuracy = accuracy_score(y_test, y_pred)
        model_accuracies[model_name] = accuracy

        print(f"Acurácia do modelo {model_name}: {accuracy:.4f}")

        # Salvar o melhor modelo
        best_models[model_name] = best_model

    # Plotar gráfico de desempenho comparativo
    plt.figure(figsize=(10, 6))
    plt.bar(model_accuracies.keys(), model_accuracies.values())
    plt.xlabel('Modelos')
    plt.ylabel('Acurácia')
    plt.title('Comparação de desempenho dos modelos')
    plt.xticks(rotation=45)
    plt.tight_layout()
    graph_filename = f"{base_name}_graph_{model_name}.png"
    graph_path = os.path.join(current_dir, graph_filename)
    plt.savefig(graph_path)
    plt.close()

    #Pickle the single best model, verifying the best and saving from best_models dict, se tiver mais algo salvo com o mesmo nome, adiciona mais um.
    best_model = max(model_accuracies, key=model_accuracies.get)
    best_model = best_models[best_model]
    if os.path.exists(f"{base_name}_best_model.pkl"):
        i = 1
        while os.path.exists(f"{base_name}_best_model_{i}.pkl"):
            i += 1
        pickle.dump(best_model, open(f"{base_name}_best_model_{i}.pkl", "wb"))
    else:
        pickle.dump(best_model, open(f"{base_name}_best_model.pkl", "wb"))
    # Salvar tabela com os valores de acurácia
    table_filename = f"{base_name}_table_{model_name}.csv"
    accuracies_df = pd.DataFrame.from_dict(model_accuracies, orient='index', columns=['Acurácia'])
    table_path = os.path.join(current_dir, table_filename)
    accuracies_df.to_csv(table_path, index=False)

    return best_models

In [50]:
# Realizar o tuning dos modelos
best_models = perform_model_tuning(X_train_tfidf, y_train, models, param_grids, 'tfidf')

Tuning do modelo Random Forest iniciado...
Fitting 5 folds for each of 216 candidates, totalling 1080 fits


KeyboardInterrupt: 